# Final project: Meta Aggregation #

### by: Ortal & Raz & Michal
### guidence: Hila

In [ ]:
!pip install shap

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.neural_network import MLPClassifier
from skmultilearn.problem_transform import ClassifierChain
from sklearn.model_selection import train_test_split
from GetProcessedData import get_question_dfs
from AggregationMethods.ConfidenceMethods import highest_average_confidence
from AggregationMethods.SurprisinglyPopular import surprisingly_pop_answer
from AggregationMethods.MajorityRule import majority_answer
from FeaturesExtraction.AnswerFeatures import AnswerF
from FeaturesExtraction.AnswerFeaturesSubgroups import AnswerSubF
from FeaturesExtraction.ConfidenceFeatures import ConfidenceF
from FeaturesExtraction.ConfidenceFeaturesSubgroups import ConfidenceSubF
from FeaturesExtraction.PredictionFeatures import PredictionsF

In [4]:
#libraries
import sys
import os

module_path = os.path.abspath(os.path.join(os.pardir, os.pardir))
if module_path not in sys.path:
    sys.path.append(module_path)

from ProjectFramework.GetProcessedData import get_question_dfs
from ProjectFramework.Pipeline.MultiLabelPipeline import create_data_df

#import ProjectFramework.Pipeline.MultiLabelPipeline.MultiLabelPipeline.create_data_df
import numpy as np 
import pandas as pd 
import os
import json 
import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use('ggplot')
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold, KFold
from sklearn.model_selection import LeaveOneOut
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification
from sklearn.metrics import accuracy_score,confusion_matrix,classification_report
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split

[[1 1 1]
 [1 1 1]
 [0 1 1]
 [1 1 1]
 [0 1 1]
 [0 1 1]
 [1 1 1]
 [0 1 1]
 [1 1 1]
 [1 1 1]
 [1 1 1]
 [1 1 1]
 [0 1 1]
 [0 1 0]
 [0 1 1]
 [1 1 1]
 [1 1 1]
 [1 1 1]
 [1 1 1]
 [1 1 1]
 [0 1 1]
 [1 1 1]
 [1 1 1]
 [0 1 1]
 [1 1 1]
 [1 1 1]
 [1 1 1]
 [1 1 1]
 [1 1 1]
 [0 1 1]
 [0 1 1]
 [0 1 1]
 [0 1 0]
 [0 1 1]
 [1 1 1]
 [1 1 1]
 [1 1 1]
 [0 0 1]
 [1 1 1]
 [1 1 1]
 [1 1 1]
 [1 1 1]
 [0 1 1]
 [1 1 1]
 [1 1 1]
 [0 1 1]
 [1 1 1]
 [1 1 1]
 [0 1 0]
 [1 1 1]
 [0 1 1]
 [1 1 1]
 [1 1 1]
 [0 1 1]
 [0 1 1]
 [0 1 1]
 [1 1 1]
 [1 1 1]
 [1 1 1]
 [0 1 0]
 [1 1 1]
 [1 1 1]
 [1 1 1]
 [1 1 1]
 [1 1 1]
 [0 1 1]
 [1 1 1]
 [1 1 1]
 [1 1 1]
 [0 1 1]
 [1 1 1]
 [0 1 1]
 [1 1 0]
 [0 1 1]
 [1 1 1]
 [1 1 1]
 [0 1 1]
 [1 1 1]
 [0 1 0]
 [0 1 0]
 [0 1 1]
 [1 1 1]
 [0 1 1]
 [1 1 1]
 [1 1 1]
 [0 1 1]
 [0 1 1]
 [1 1 1]
 [1 1 1]
 [1 1 1]
 [0 1 1]
 [1 1 1]
 [0 1 1]
 [1 1 1]
 [0 1 0]
 [0 1 1]
 [0 1 1]
 [0 1 0]
 [1 1 1]
 [1 1 1]
 [0 1 1]
 [0 1 1]
 [0 1 1]
 [1 1 1]]
0.5480769230769231
training
predicting
0.7403846153846154
Test 

In [7]:
# get compleate data frame with features:

DATA = create_data_df()

DATA.head()
#X = DATA.drop("labels")
#y = DATA["Labels"]

single positional indexer is out-of-bounds


  File "C:\Users\school & work\PycharmProjects\Final_Project_SISE_BGU\ProjectFramework\Pipeline\MultiLabelPipeline.py", line 114, in create_data_df
    correct_answer = str(df[df['Class'] == 1]['Answer'].iloc[0])
  File "c:\python38\lib\site-packages\pandas\core\indexing.py", line 1768, in __getitem__
    return self._getitem_axis(maybe_callable, axis=axis)
  File "c:\python38\lib\site-packages\pandas\core\indexing.py", line 2138, in _getitem_axis
    self._validate_integer(key, axis)
  File "c:\python38\lib\site-packages\pandas\core\indexing.py", line 2063, in _validate_integer
    raise IndexError("single positional indexer is out-of-bounds")


,,AS_distribution_most_pop_ans,AS_entropy_distance_highest_lowest,AS_has_pop_ans_changed,AS_std_distance_highest_lowest,AS_var_distance_highest_lowest,A_above50%,A_distance1-2,A_distance1-last,A_entropy,...,P_highest_mean,P_highest_std,P_highest_var,P_low_con_high_p,P_lower_then_votes,P_lowest_mean,P_lowest_std,P_lowest_var,P_predicition,SP
0,1.0,0.004691,0.218756,0.0,1.206648,6.181818,1.0,7.091873,19.091873,2.115402,...,0.181957,0.231198,0.053453,4.0,67.0,0.043804,0.047476,0.002254,2.0,1.0
1,1.0,0.001519,0.126706,1.0,1.325765,11.500000,1.0,4.327382,21.327382,1.262139,...,0.368878,0.240059,0.057628,4.0,98.0,0.128061,0.136663,0.018677,0.0,0.0
2,0.0,0.001728,0.134766,1.0,0.934701,11.666667,1.0,11.622996,34.622996,1.115028,...,0.462637,0.312786,0.097835,0.0,82.0,0.036703,0.058216,0.003389,4.0,0.0
3,0.0,0.004535,0.304301,0.0,1.123525,4.000000,1.0,-2.242641,8.757359,1.933947,...,0.199219,0.234443,0.054963,4.0,57.0,0.079687,0.101195,0.010241,3.0,1.0
4,0.0,0.008822,0.263845,0.0,3.115926,16.500000,1.0,-4.088723,8.911277,1.302358,...,0.310000,0.271714,0.073829,3.0,70.0,0.151429,0.171125,0.029284,0.0,0.0


In [ ]:
# split data into train and test using regular split

# create training and testing vars
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2)
print X_train.shape, y_train.shape
print X_test.shape, y_test.shape



In [ ]:
# split data into train and test using k-fold cross validation

kf = KFold(n_splits=2, random_state=None, shuffle=False) # Define the split - into 2 folds 
kf.get_n_splits(X) # returns the number of splitting iterations in the cross-validator

for train_index, test_index in kf.split(X):
    # print(“TRAIN:”, train_index, “TEST:”, test_index)
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    ## HERE - run moddels
    

In [ ]:
# split data into train and test using leave one out validation

loo = LeaveOneOut()
loo.get_n_splits(X)

for train_index, test_index in loo.split(X):
    # print("TRAIN:", train_index, "TEST:", test_index)
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    # print(X_train, X_test, y_train, y_test)
    ## HERE - run moddels



In [ ]:
from sklearn.model_selection import cross_val_score, cross_val_predict
from sklearn import metrics

# Perform k-fold cross validation
k = 10
scores = cross_val_score(model, x, y, cv=k)
print “Cross-validated scores:”, scores
    
# Make cross validated predictions
predictions = cross_val_predict(model, df, y, cv=k)
plt.scatter(y, predictions)

## present accuracy

In [ ]:
## present accuracy
accuracy_list = []

## add all models accuracy to list
accuracy_list.append(accuracy_score(nb_predict,y_test))
accuracy_list.append(accuracy_score(knn_predict,y_test))
accuracy_list.append(accuracy_score(rfc_predict,y_test))
accuracy_list.append(accuracy_score(logreg_predict,y_test))
accuracy_list.append(accuracy_score(svm_predict,y_test))
accuracy_list.append(accuracy_score(adc_predict,y_test))

accuracy_dataframe = pd.DataFrame(columns=["Model","Accuracy"])
model_list = ['Naive_Bayes','KNN','Random_Forest','Logistic_Reg','SVM','Ada_Boost']

for i in range(len(accuracy_list)):
    accuracy_dataframe.loc[i] = model_list[i], accuracy_list[i]

accuracy_dataframe

## shap feature evaluation

In [ ]:
# load JS vis
shap.initjs() 

## the best model:
model = rt_model

# create a tree explainer for Random Forest
explainer = shap.TreeExplainer(model)

#sample 100 observations from the test data
observations = X_test.sample(100, random_state=42)

#create an explainer
shap_values = explainer.shap_values(observations)

#generate a summary plot
shap.summary_plot(shap_values[1], features=observations)

## confusion matrix

In [ ]:
#confusion matrix visualization

def plot_confusion_matrix(test_Y, predict_y):
    C = confusion_matrix(test_Y, predict_y)
    A =(((C.T)/(C.sum(axis=1))).T)
    B =(C/C.sum(axis=0))
    plt.figure(figsize=(20,4))
    labels = [1,2]
    cmap=sns.light_palette("#45c6c4")
    plt.subplot(1, 3, 1)
    sns.heatmap(C, annot=True, cmap=cmap, fmt=".3f", xticklabels=labels, yticklabels=labels)
    plt.xlabel('Predicted Class')
    plt.ylabel('Original Class')
    plt.title("Confusion matrix")
    plt.subplot(1, 3, 2)
    sns.heatmap(B, annot=True, cmap=cmap, fmt=".3f", xticklabels=labels, yticklabels=labels)
    plt.xlabel('Predicted Class')
    plt.ylabel('Original Class')
    plt.title("Precision matrix")
    plt.subplot(1, 3, 3)
    sns.heatmap(A, annot=True, cmap=cmap, fmt=".3f", xticklabels=labels, yticklabels=labels)
    plt.xlabel('Predicted Class')
    plt.ylabel('Original Class')
    plt.title("Recall matrix")
    plt.show()

plot_confusion_matrix(y_test, rfc_predict)